In [16]:
1+1
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import json
from datetime import date, datetime, timedelta
import time
import os
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
import pathlib
from config import ACCESS_KEY,SECRET_KEY
import boto3

ModuleNotFoundError: No module named 'config'

In [19]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import json
from datetime import date, datetime, timedelta
import time
import os
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
import pathlib
# from config import ACCESS_KEY,SECRET_KEY
import boto3


# Generating Order
# API Key stored as an env variable
PL_API_KEY = 'd1522b0573f548ea8233d09b9a917b74'
PLANET_API_KEY = PL_API_KEY #os.getenv('PL_API_KEY')
orders_v2_url = 'https://api.planet.com/compute/ops/orders/v2'

# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}


item_type = "PSScene4Band"

In [20]:
waypoint_data = {
    'Waypoint' : ['Orangi River', 'Mara River', 'Sand River', 'Grumeti River', 'Lake Masek', 
                  'Lake Magadi', 'Lake Empakaai', 'Lake Magadi 2', 'Mbalageti River', 'Ruwana River', 'Talek River'],
    'latitude' : [-2.302313, -1.562928, -1.595733, -2.249034, -3.024818, -3.202214, -2.915433, -2.656248, -2.603015, -2.044819, -1.416096],
    'longitude' : [34.830777, 34.997068, 35.069241, 34.486842, 35.038474, 35.536431, 35.841355, 34.788239, 34.720511, 34.230374, 35.097661]
}


waypoint_df = pd.DataFrame(waypoint_data)
waypoint_gdf = gpd.GeoDataFrame(waypoint_df, geometry=gpd.points_from_xy(waypoint_df.longitude, waypoint_df.latitude))

# Applying WGS84 to the CRS
waypoint_gdf.crs = {'init' :'epsg:4326'} 

# Converting geodataframe to Meters from Lat/Long
# Allows for square buffer to be applied (450m)
point_gdf_m = waypoint_gdf.to_crs(epsg=3395)

# Applying the buffer, cap_style = 3 --> Square Buffer
buffer = point_gdf_m.buffer(450, cap_style=3)

# Convert buffer back to WGS84 Lat/Long
buffer_wgs84 = buffer.to_crs(epsg=4326)

# Merging GDF and DF to get the Waypoint names
joined_buffer_wgs84 = pd.concat([waypoint_df,buffer_wgs84], axis=1)
joined_buffer_wgs84 = joined_buffer_wgs84.rename(columns = {0:'polygon'}).set_geometry('polygon')
joined_buffer_wgs84_drop = joined_buffer_wgs84.drop(['geometry'], axis=1)
joined_buffer_wgs84_json = joined_buffer_wgs84_drop.to_json()

# transforming to json for inclusion into Planet API
buffer_wgs84_json_parsed = json.loads(joined_buffer_wgs84_json)
buffer_wgs84_json_api = buffer_wgs84_json_parsed['features']#[0]['geometry']['coordinates']


today = datetime.isoformat(datetime.utcnow())+'Z'#(datetime.today())
start_date = datetime.isoformat(datetime.utcnow() - timedelta(7)) + 'Z'

/Users/piper/anaconda2/envs/py36/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [21]:
# Getting Image ID's for each waypoint that has the analytic_sr dataset 
# Having to ping the Planet V1 API to return the image id's for our required filter
# Filter variables include: Center Coordinate, Date Range, Cloud Cover, Item Type and Asset Type


def build_order(index):
    geojson_geometry = {
    "type": "Point",
    "coordinates": [
        index['properties']['longitude'], index['properties']['latitude']
        ]
    }

    # get images that overlap with our AOI 
    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geojson_geometry
    }

    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": start_date,
        "lte": today
      }
    }

    # only get images which have <10% cloud coverage
    cloud_cover_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": 0.1
      }
    }

    # combine our geo, date, cloud filters
    combined_filter = {
      "type": "AndFilter",
      "config": [geometry_filter, date_range_filter, cloud_cover_filter]
    }
    
    # API request object
    search_request = {
        "interval": "day",
        "item_types": [item_type],
        "asset_types" : "analytic_sr",
        "filter": combined_filter
    }
    
    search_result = \
      requests.post(
        'https://api.planet.com/data/v1/quick-search',
    #     'https://api.planet.com/data/v2',
        auth=HTTPBasicAuth(PLANET_API_KEY, ''),
        json=search_request)

    return search_result



id_list = []

for index in buffer_wgs84_json_api:
    waypoint = index["properties"]["Waypoint"]
    order = build_order(index).json()['features']
#     print(len(order))


# appending Image ID to `joined_buffer_wgs84_drop_merge` if the analytic_sr is available
# Will only return image id's that meet this requirement.
    for i in order:
        #print(order)
        if "assets.analytic_sr:download" in order[0]["_permissions"]:
            id_list.append((waypoint,order[0]["id"]))

id_list = list(set(id_list))

In [22]:
id_list

[('Orangi River', '20200311_074225_103b'),
 ('Lake Magadi 2', '20200308_084112_13_1065'),
 ('Mbalageti River', '20200311_074230_103b'),
 ('Sand River', '20200312_064310_1053'),
 ('Mara River', '20200312_064311_1053'),
 ('Ruwana River', '20200308_074723_1009')]

In [23]:
# Merging image id's to the dataframe to maintain continuity

image_ids = pd.DataFrame(np.asarray(id_list))
image_ids.rename(columns = {0:'Waypoint', 1:'Image_ID'}, inplace = True) 

joined_buffer_wgs84_drop_merge = pd.merge(image_ids, joined_buffer_wgs84_drop, on='Waypoint')
joined_buffer_wgs84_drop_merge

,Waypoint,Image_ID,latitude,longitude,polygon
0,Orangi River,20200311_074225_103b,-2.302313,34.830777,"POLYGON ((34.83482 -2.29825, 34.83482 -2.30638..."
1,Lake Magadi 2,20200308_084112_13_1065,-2.656248,34.788239,"POLYGON ((34.79228 -2.65218, 34.79228 -2.66031..."
2,Mbalageti River,20200311_074230_103b,-2.603015,34.720511,"POLYGON ((34.72455 -2.59895, 34.72455 -2.60708..."
3,Sand River,20200312_064310_1053,-1.595733,35.069241,"POLYGON ((35.07328 -1.59166, 35.07328 -1.59980..."
4,Mara River,20200312_064311_1053,-1.562928,34.997068,"POLYGON ((35.00111 -1.55886, 35.00111 -1.56700..."
5,Ruwana River,20200308_074723_1009,-2.044819,34.230374,"POLYGON ((34.23442 -2.04075, 34.23442 -2.04889..."


In [25]:
# Converting list of tuple polygons to list of lists polygons
# This step is necessary to pull the Geometry from `joined_buffer_wgs84_drop_merge`
# and convert to a list of lists...appending to `joined_buffer_wgs84_drop_merge`.

def coord_lister(geom):
    coords = list(geom.exterior.coords)
    return (coords)

coordinates = joined_buffer_wgs84_drop_merge.polygon.apply(coord_lister)
res = []
for poly in coordinates:
    res_2 = list(map(list, poly)) 
    res.append(res_2)

joined_buffer_wgs84_drop_merge['poly_list'] = res
# joined_buffer_wgs84_drop_merge will be used as the basis for all remaining functions
joined_buffer_wgs84_drop_merge

,Waypoint,Image_ID,latitude,longitude,polygon,poly_list
0,Orangi River,20200311_074225_103b,-2.302313,34.830777,"POLYGON ((34.83482 -2.29825, 34.83482 -2.30638...","[[34.83481941877853, -2.298246660514488], [34...."
1,Lake Magadi 2,20200308_084112_13_1065,-2.656248,34.788239,"POLYGON ((34.79228 -2.65218, 34.79228 -2.66031...","[[34.79228141877853, -2.652182761619736], [34...."
2,Mbalageti River,20200311_074230_103b,-2.603015,34.720511,"POLYGON ((34.72455 -2.59895, 34.72455 -2.60708...","[[34.724553418778534, -2.598949585965823], [34..."
3,Sand River,20200312_064310_1053,-1.595733,35.069241,"POLYGON ((35.07328 -1.59166, 35.07328 -1.59980...","[[35.07328341877854, -1.5916649326258994], [35..."
4,Mara River,20200312_064311_1053,-1.562928,34.997068,"POLYGON ((35.00111 -1.55886, 35.00111 -1.56700...","[[35.00111041877854, -1.558859867631091], [35...."
5,Ruwana River,20200308_074723_1009,-2.044819,34.230374,"POLYGON ((34.23442 -2.04075, 34.23442 -2.04889...","[[34.23441641877854, -2.040751958243945], [34...."


In [26]:
# Creating the URLs to activate the images...prevents latency during download
id0_url = []
for id0 in joined_buffer_wgs84_drop_merge['Image_ID']:
    id0_url_2 = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)
    id0_url.append(id0_url_2)

id0_url


['https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200311_074225_103b/assets',
 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200308_084112_13_1065/assets',
 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200311_074230_103b/assets',
 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200312_064310_1053/assets',
 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200312_064311_1053/assets',
 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200308_074723_1009/assets']

In [27]:
# Returns JSON metadata for assets in this ID. 
# Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = []
for link in id0_url:
#     print(link)
    result1 = \
      requests.get(
        link,
        auth=HTTPBasicAuth(PLANET_API_KEY, '')
      )
    result.append(result1)

# Parse out useful links

links = []
activation_link = []

# Getting Result Links
for r in result:
    links1 = r.json()[u"analytic_sr"]["_links"]
    links.append(links1)

# Generating a list of activation links    
for l in links:
    activation_link1 = l["activate"]
    activation_link.append(activation_link1)
activation_link

# Request activation of the 'visual' asset:
for a in activation_link:
    activate_result = \
    requests.get(a,auth=HTTPBasicAuth(PLANET_API_KEY, ''))

In [29]:
activate_result

<Response [204]>


In [30]:
# Building the order lists starting with the product information

single_product = []
for image_id in joined_buffer_wgs84_drop_merge['Image_ID']:
    single_product_2 = [
        {
          'item_ids': [image_id], 
          'item_type': 'PSScene4Band',
          'product_bundle': 'analytic_sr'
        }
    ]
    single_product.append(single_product_2)
print(single_product)


[[{'item_ids': ['20200311_074225_103b'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic_sr'}], [{'item_ids': ['20200308_084112_13_1065'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic_sr'}], [{'item_ids': ['20200311_074230_103b'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic_sr'}], [{'item_ids': ['20200312_064310_1053'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic_sr'}], [{'item_ids': ['20200312_064311_1053'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic_sr'}], [{'item_ids': ['20200308_074723_1009'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic_sr'}]]


In [31]:
# Setting the clipping boundaries

clip = []
for polygon in joined_buffer_wgs84_drop_merge['poly_list']:
    clip_aoi = {
        'type':'Polygon',
        'coordinates': [polygon] 
    }

# define the clip tool
    clip_2 = {
        'clip': {
            'aoi': clip_aoi
        }
    }
    clip.append(clip_2)

In [32]:
# create an order request with the clipping tool
request_clip = []

for p, c in zip(single_product, clip):
    request_clip_2 = {
        'name': 'just clip',
        'products': p, #single_product,
        'tools': [c]
    }
    request_clip.append(request_clip_2)

In [34]:
# Creating the url for clipping 
def place_order(request, auth):
    response = requests.post(orders_v2_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if not response.ok:
        raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    order_url = orders_v2_url + '/' + order_id
    return order_url


clip_order_url = []
for req_c in request_clip:
    clip_order_url2 = place_order(request_clip[1], auth)
    clip_order_url.append(clip_order_url2)
clip_order_url


<Response [202]>
bf801ac8-c321-482f-91f2-9a722ce237a0
<Response [202]>
a942157b-7879-4759-b174-71e03e1b299e
<Response [202]>
e4d6c574-d18d-4f7c-84fe-28e1b8eef30b
<Response [202]>
9fb9b4be-d286-4019-b526-ccf2f5673071
<Response [202]>
2a792796-defd-4411-89aa-0c066bf3ca69
<Response [202]>
2b48616c-9d02-466e-945d-400eb2777384


['https://api.planet.com/compute/ops/orders/v2/bf801ac8-c321-482f-91f2-9a722ce237a0',
 'https://api.planet.com/compute/ops/orders/v2/a942157b-7879-4759-b174-71e03e1b299e',
 'https://api.planet.com/compute/ops/orders/v2/e4d6c574-d18d-4f7c-84fe-28e1b8eef30b',
 'https://api.planet.com/compute/ops/orders/v2/9fb9b4be-d286-4019-b526-ccf2f5673071',
 'https://api.planet.com/compute/ops/orders/v2/2a792796-defd-4411-89aa-0c066bf3ca69',
 'https://api.planet.com/compute/ops/orders/v2/2b48616c-9d02-466e-945d-400eb2777384']

In [39]:
for c in clip_order_url:
    r = requests.get(c, auth=auth)
    print(r.json())
    time.sleep(1)


{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/bf801ac8-c321-482f-91f2-9a722ce237a0', 'results': [{'delivery': 'success', 'expires_at': '2020-03-16T03:58:29.075Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1ODQzMzExMDksInN1YiI6ImtnVUtGWWxWemlCMnNoWm5tYVdLa3BOR25HOEljYVl1TWNHcnUvT2dGWXdWWThjemU4NHpGWWRaT1RFN3BFTXFXei8wbzBUeHdRYjVIMy9JdmkySWRnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS9iZjgwMWFjOC1jMzIxLTQ4MmYtOTFmMi05YTcyMmNlMjM3YTAvMS9tYW5pZmVzdC5qc29uP0V4cGlyZXM9MTU4NDMzMTEwOVx1MDAyNkdvb2dsZUFjY2Vzc0lkPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb21cdTAwMjZTaWduYXR1cmU9VUt6Sk1WU1UlMkZGcEclMkJHZUtoaTdTU1BCMGV3UGlyNCUyQjZXR2c5Y0NZaGpPbHR3NkFsQjFSRE9RU1NVZnN2Y2VRa3gzYXRucnpBSE9WbmFtU3l6elV

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/a942157b-7879-4759-b174-71e03e1b299e', 'results': [{'delivery': 'success', 'expires_at': '2020-03-16T03:58:29.084Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1ODQzMzExMDksInN1YiI6InNkVlNhL1M5Vy9pMmw2dnNLUFdsZ1VnbFBKWVZKb1VBelJHRzZVR0VudHVZM296V2VPOHorSVFpdEVJK3YrYXBsQVdTYno1ZTRKdVdBQlUyL3RXa0RRPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS9hOTQyMTU3Yi03ODc5LTQ3NTktYjE3NC03MWUwM2UxYjI5OWUvMS9maWxlcy8yMDIwMDMwOF8wODQxMTJfMTNfMTA2NV8zQl9BbmFseXRpY01TX21ldGFkYXRhX2NsaXAueG1sP0V4cGlyZXM9MTU4NDMzMTEwOVx1MDAyNkdvb2dsZUFjY2Vzc0lkPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb21cdTAwMjZTaWduYXR1cmU9WlFMMmVaJTJGeHg1WG9BbUdHdzVpYVlONlNSQ1FUVHVIRUk0czVrbXZ

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/e4d6c574-d18d-4f7c-84fe-28e1b8eef30b', 'results': [{'delivery': 'success', 'expires_at': '2020-03-16T03:58:29.072Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1ODQzMzExMDksInN1YiI6InQ0aFR1WEY1UWN6SllZcWhlV2thVk1HS2p6ZHcvZGNKMDl0ZFhpcDZObjJ5UFc3SWM5YXRsVUNrUkJTTFJNR0tUYlNkbVJVRVFGT0U3cjZCQUg4bjhnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS9lNGQ2YzU3NC1kMThkLTRmN2MtODRmZS0yOGUxYjhlZWYzMGIvMS9tYW5pZmVzdC5qc29uP0V4cGlyZXM9MTU4NDMzMTEwOVx1MDAyNkdvb2dsZUFjY2Vzc0lkPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb21cdTAwMjZTaWduYXR1cmU9aTJVTTFhSmlXSHc0QndPJTJGQ1dIdnJuejVJeDFpeXZZaUVZa1hEckJ5Z2ljWjZBTjJRZ29CVndNZ2RpOEYlMkZSbjNsOXNkdE9udiUyRjgzVGVpRGxIYTh

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/9fb9b4be-d286-4019-b526-ccf2f5673071', 'results': [{'delivery': 'success', 'expires_at': '2020-03-16T03:58:29.078Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1ODQzMzExMDksInN1YiI6IlFyS0g5eDBLVU5jVy9KQmUwQVQyMXZtdEk3QXpUQ0VDckUwb0o4dEVyTWhnZnMyQjBBaTRueUZsWVpCZElocnpRZURwK1VZNTY2ZjlOSW9HUkcrZ1JRPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS85ZmI5YjRiZS1kMjg2LTQwMTktYjUyNi1jY2YyZjU2NzMwNzEvMS9tYW5pZmVzdC5qc29uP0V4cGlyZXM9MTU4NDMzMTEwOVx1MDAyNkdvb2dsZUFjY2Vzc0lkPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb21cdTAwMjZTaWduYXR1cmU9alBnYjU2ZzRQeUZqcGwzVGZ2WnlkRDFjSkxmSGoxWmYlMkZKdDJXSVhWenNoSGZQT1FpUnljaXNtaFVXUExMTzRpajk0N005Q1dqTDMxYjlIcXFQRWd2Mm1

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/2a792796-defd-4411-89aa-0c066bf3ca69', 'results': [{'delivery': 'success', 'expires_at': '2020-03-16T03:58:29.085Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1ODQzMzExMDksInN1YiI6Im9XY056MGc4NmdmaUJIL0YzaGdncmFYeVVOVEZRNjVvQjB3UzlVWkRqTmVIdUhDQ2t3TTA0R0FqSTRkeFg4WitzZmJVY3duNjZzeElWeHo0Q1YvdWRBPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS8yYTc5Mjc5Ni1kZWZkLTQ0MTEtODlhYS0wYzA2NmJmM2NhNjkvMS9maWxlcy8yMDIwMDMwOF8wODQxMTJfMTNfMTA2NV8zQl9BbmFseXRpY01TX1NSX2NsaXAudGlmP0V4cGlyZXM9MTU4NDMzMTEwOVx1MDAyNkdvb2dsZUFjY2Vzc0lkPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb21cdTAwMjZTaWduYXR1cmU9VGk3SG1mVkt6cmlONWpoeTMwSHJ4YTJSWmF3Zzlja0tmVEFwJTJGcUg3dDVwRXp

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/2b48616c-9d02-466e-945d-400eb2777384', 'results': [{'delivery': 'success', 'expires_at': '2020-03-16T03:59:02.263Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1ODQzMzExNDIsInN1YiI6Inc4NXJGT0hPVGlOdWdSUGZpNnpyd0dCZThBYUc2bVBDK3pTbkhpTTh2Z3ZxQjYra2wvZHoxWDZjcUxSZ09YSGsvVTN4ZlNjR3kvem44ZWxQc3dKbEl3PT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS8yYjQ4NjE2Yy05ZDAyLTQ2NmUtOTQ1ZC00MDBlYjI3NzczODQvMS9maWxlcy8yMDIwMDMwOF8wODQxMTJfMTNfMTA2NV9tZXRhZGF0YS5qc29uP0V4cGlyZXM9MTU4NDMzMTE0Mlx1MDAyNkdvb2dsZUFjY2Vzc0lkPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb21cdTAwMjZTaWduYXR1cmU9bWclMkZ0MW0yb1U5QURMbTNZcFNLS2wwM0xEJTJCNmNXUnFEaVc0Mkx4TiUyRlRBVmpOclNqSk9Ob1N

In [40]:
r.json()

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/2b48616c-9d02-466e-945d-400eb2777384',
  'results': [{'delivery': 'success',
    'expires_at': '2020-03-16T03:59:02.263Z',
    'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1ODQzMzExNDIsInN1YiI6Inc4NXJGT0hPVGlOdWdSUGZpNnpyd0dCZThBYUc2bVBDK3pTbkhpTTh2Z3ZxQjYra2wvZHoxWDZjcUxSZ09YSGsvVTN4ZlNjR3kvem44ZWxQc3dKbEl3PT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS8yYjQ4NjE2Yy05ZDAyLTQ2NmUtOTQ1ZC00MDBlYjI3NzczODQvMS9maWxlcy8yMDIwMDMwOF8wODQxMTJfMTNfMTA2NV9tZXRhZGF0YS5qc29uP0V4cGlyZXM9MTU4NDMzMTE0Mlx1MDAyNkdvb2dsZUFjY2Vzc0lkPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb21cdTAwMjZTaWduYXR1cmU9bWclMkZ0MW0yb1U5QURMbTNZcFNLS2wwM0xEJTJCNmNXUnFEaVc0Mkx4TiUyRlRBVmpOc

In [41]:
# Downloading from each clip order url
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join('data', n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))


# Process sleeping to allow for activation, staging, and processing of imagery
time.sleep(1)

# Downloading the orders
for clip_order in clip_order_url:
    download_order(clip_order, auth)
    
print('Got em all')

<Response [200]>
5 items to download
downloading bf801ac8-c321-482f-91f2-9a722ce237a0/1/manifest.json to data/bf801ac8-c321-482f-91f2-9a722ce237a0/1/manifest.json
downloading bf801ac8-c321-482f-91f2-9a722ce237a0/1/files/20200308_084112_13_1065_3B_AnalyticMS_DN_udm_clip.tif to data/bf801ac8-c321-482f-91f2-9a722ce237a0/1/files/20200308_084112_13_1065_3B_AnalyticMS_DN_udm_clip.tif
downloading bf801ac8-c321-482f-91f2-9a722ce237a0/1/files/20200308_084112_13_1065_3B_AnalyticMS_metadata_clip.xml to data/bf801ac8-c321-482f-91f2-9a722ce237a0/1/files/20200308_084112_13_1065_3B_AnalyticMS_metadata_clip.xml
downloading bf801ac8-c321-482f-91f2-9a722ce237a0/1/files/20200308_084112_13_1065_3B_AnalyticMS_SR_clip.tif to data/bf801ac8-c321-482f-91f2-9a722ce237a0/1/files/20200308_084112_13_1065_3B_AnalyticMS_SR_clip.tif
downloading bf801ac8-c321-482f-91f2-9a722ce237a0/1/files/20200308_084112_13_1065_metadata.json to data/bf801ac8-c321-482f-91f2-9a722ce237a0/1/files/20200308_084112_13_1065_metadata.json
<

In [ ]:
# s3_resource = boto3.resource('s3')
# s3_resource.meta.client.upload_file(
#     Filename='/Users/jasonabaker/W210/Project/Planet_API/Waypoint_Scripts_2/data/e59ba56a-c72a-4f49-b680-273e243f8fd1/1/files/20200305_163004_0f44_3B_AnalyticMS_SR_clip.tif', Bucket='w210-planet-data-api',
#     Key= '20200305_163004_0f44_3B_AnalyticMS_SR_clip.tif')